Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [1]:
from collections import Counter
import numpy as np
from IPython.display import clear_output    # Useful to clear notebook output during program execution

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [3]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0

def first_move(board):
    if not np.any(board):
        return 1
    return 0

def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## Montecarlo Evaluation

In [4]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0

MONTECARLO_SAMPLES = 50 # Reduced from 100 to 50, personally experience similar results but faster
MONTECARLO_SAMPLES_FAST = 25 # This is used in monte carlo tree seach

def montecarlo(board, player, MCTS=0):
    n_simulations = MONTECARLO_SAMPLES
    if MCTS == 1:
        n_simulations = MONTECARLO_SAMPLES_FAST
    cnt = Counter(_mc(np.copy(board), player) for _ in range(n_simulations))
    return (cnt[1] - cnt[-1]) / n_simulations


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

## MinMax: Alpha-Beta + Max Depth

In [5]:
'''
Author: Alessandro Versace

MinMax Alpha-Beta approach using a max depth to early cut branches evaluation.
This is a Connect 4 AI vs Player match where the AI is always player 1. 
'''

MAX_DEPTH = 2 # Changes the MinMax depth (The higher the slower) (I suggest 2 or 3, not more)

def eval_board_minmax(board):
    if four_in_a_row(board, 1):
        # AI won
        return 1
    elif four_in_a_row(board, -1):
        # You won
        return -1
    else:
        return 0

def is_leaf_node(board, player):
    return four_in_a_row(board, player) or not valid_moves(board)

def minmax(board, player, alpha, beta, depth):

    possible = valid_moves(board)
    leaf = is_leaf_node(board, player)

    if depth == 0 or leaf:
        if leaf:
            if four_in_a_row(board, player):
                return None, 1000000000000000
            elif four_in_a_row(board, -player):
                return None, -1000000000000000
            else:
                return None, 0
        else:
            return None, eval_board(board, player)  # Here I use the montecarlo evaluation provided by Professor

    if possible:
        np.random.shuffle(possible) # Add some randomness
    
    if player == 1: # 1 is always maximizing player
        value = np.NINF
        column = np.random.choice(possible)
        for col in possible:
            play(board, col, player)
            _, score = minmax(board, -player, alpha, beta, depth-1)
            take_back(board, col)
            if score > value:
                value = score
                column = col
            alpha = max(value, alpha)

            if alpha >= beta:
                break
        return column, value


    else:   # minimizing player
        value = np.Inf
        column = np.random.choice(possible)
        for col in possible:
            play(board, col, player)
            _, score = minmax(board, -player, alpha, beta, depth-1)
            take_back(board, col)
            if score < value:
                value = score
                column = col
            beta = min(value, beta)

            if alpha >= beta:
                break
        return column, value
             

## Monte Carlo Tree Search

In [6]:
'''
Author: Alessandro Versace

Monte Carlo Tree Search approach, by default it starts by selecting a random number of nodes from the current state
then goes ahead with expansions and simulations and finally backup.
You can experience much higher performances (in terms of speed) in respect to the MinMax preceding, but sometimes happens it makes bad decisions.
This is a Connect 4 AI vs Player match where the AI is always player 1. 
'''

def mc_tree_search(board, player, depth=1):
    
    possible = valid_moves(board)
    leaf = is_leaf_node(board, player)

    if depth == 0 or leaf:
        if leaf:
            if four_in_a_row(board, player):
                return 0, 1000000000000000
            elif four_in_a_row(board, -player):
                return 0, -1000000000000000
            else:
                return 0, 0
        else:
            return 1, montecarlo(board, player, MCTS=1) # SIMULATIONS

    evaluations = []
    n_samples = len(possible) # Useful for initial expantions if we start with a depth > 1 (similar to initial minmax steps)
    if depth == 1:  # Before starting Monte Carlo Search select some random nodes to expand // SELECTION
        np.random.shuffle(possible)
        if possible and len(possible) > 3:
            n_samples = np.random.randint(1, high=len(possible)*3//4)    # Select a random number of nodes to expand
        elif possible and len(possible) <= 3 and len(possible) > 1:
            n_samples = np.random.randint(1, high=len(possible)//2)
        elif possible and len(possible) == 1:
            n_samples = 1


    if player == 1: # 1 is always maximizing player
        for i, col in enumerate(possible):
            if i == n_samples:
                break
            play(board, col, player)
            _, score = mc_tree_search(board, -player, depth-1) # EXPANSIONS
            take_back(board, col)

            if score == 1000000000000000:   # Early stopping searches
                return col, score

            evaluations.append((col, score))    # Backup
        return max(evaluations, key=lambda e: e[1])

    else:   # minimizing player
        for i, col in enumerate(possible):
            if i == n_samples:
                break
            play(board, col, player)
            _, score = mc_tree_search(board, -player, depth-1)
            take_back(board, col)

            if score == -1000000000000000:
                return col, score

            evaluations.append((col, score))
        return min(evaluations, key=lambda e: e[1])

## Play

In [7]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
print(board)

# Insert here your moves for starting with a predefined configuration.
# AI is always player 1
# The Player (you) is always player -1
####
# You can swap between MINMAX and MONTE CARLO TREE SEARCH below inside the while.
# Uncomment minmax() and comment mc_tree_search() lines or vice versa.
# Enjoy!!
####

player = 1
while not four_in_a_row(board, 1) and not four_in_a_row(board, -1) and valid_moves(board):
    if first_move(board):
        print('Selected column 2 (starting from 0 from top to bottom). Sometimes the board isnt shown but here is my move.\n')
        play(board, 2, player)  # Opening if needed
    else:
        # best_move, eval = minmax(board, player, np.NINF, np.Inf, MAX_DEPTH)
        best_move, eval = mc_tree_search(board, player, depth=2) 
        # On mc_tree_search() you can use depth=1, performance are going to increase a lot,
        # but it's not certain AI will make the right choice
        # because it will work in full probabilistic environment
        
        print(f'AI inserted in this column: {best_move}.\n(if you see this is because of a problem with output cells, continue to play knowing AI position written)\n')
        if best_move == None:
            break
        play(board, best_move, player)

    clear_output(wait=True) # Useful to clear notebook output during program execution

    print(board)

    if not four_in_a_row(board, 1) and not four_in_a_row(board, -1) and valid_moves(board):
        while True:
            try: 
                human_col = int(input("\nTake your time and tell me the column if which you want to play:\n(Columns starts from 0 from top to bottom)"))
            except ValueError:
                print('Write a valid number pls...')
                continue
            if human_col >= 0 and human_col <= 6:
                play(board, human_col, -player)
                break
            else:
                print('Write a valid column number pls...')
    else:
        break
    
    print(board)

print(f'\n{board}')
won = eval_board_minmax(board)
if won:
    print(f'Player {won} won! Congratulations!')
else:
    print('Draw.')

[[-1  0  0  0  0  0]
 [ 1  1  1  0  0  0]
 [ 1 -1  1 -1  0  0]
 [-1  1  1  1 -1  0]
 [ 1 -1  1  0  0  0]
 [-1  0  0  0  0  0]
 [-1 -1  1 -1  0  0]]

[[-1  0  0  0  0  0]
 [ 1  1  1  0  0  0]
 [ 1 -1  1 -1  0  0]
 [-1  1  1  1 -1  0]
 [ 1 -1  1  0  0  0]
 [-1  0  0  0  0  0]
 [-1 -1  1 -1  0  0]]
Player 1 won! Congratulations!
